In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit # Aunque no se use para OOF base, puede ser útil para otras cosas
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import os
import gc
import time
import logging
import sys
import warnings
import pyarrow.parquet as pq
import psutil # Para registrar uso de memoria
import matplotlib.pyplot as plt


import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler # Necesario para cargar/usar scalers


# --- Configuración del Logging ---
LOG_FILE = 'stacking_model_progress_v3.log' # Nuevo nombre de log
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(LOG_FILE, mode='w'), # mode='w' para sobrescribir en cada ejecución
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger()

log_progress_initial = lambda message: logger.info(f"[{time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())}] {message}")

log_progress_initial("Iniciando script de modelos de stacking (con modelos base preentrenados) por clusters - v3")

# --- Supresión de Warnings ---
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning) # Ignorar UserWarning de LightGBM sobre tipos de datos
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning) # Ignorar PerformanceWarning de Pandas

# --- Configuración Global ---
# Rutas de archivos y directorios
PARQUET_FILE_PATH = '../data/gold_ventas_semanales_clustered_lgbm.parquet' # Asegúrate que esta ruta sea correcta
OUTPUT_DIR = '../models/stacking_preloaded_v3' # Directorio de salida
MODELS_DIR = os.path.join(OUTPUT_DIR, 'trained_meta_models') # Directorio para meta-modelos entrenados
PRETRAINED_MODELS_DIR = '../models/stacking/trained_models' # !! IMPORTANTE: Ruta a TUS modelos base preentrenados !!
PREDICTIONS_DIR = os.path.join(OUTPUT_DIR, 'predictions')
PLOTS_DIR = os.path.join(OUTPUT_DIR, 'prediction_plots')
METRICS_FILE = os.path.join(OUTPUT_DIR, 'all_clusters_metrics.csv')
CHECKPOINT_FILE = os.path.join(OUTPUT_DIR, 'stacking_checkpoint.txt')

# Nombres de columnas
TARGET_COL = 'weekly_volume'
CLUSTER_COL = 'cluster_label'
DATE_COL = 'week'
SERIES_ID_COLS = ['establecimiento', 'material'] # Identificadores únicos de series temporales

# Parámetros del Modelo y Procesamiento
N_TEST_WEEKS = 12  # Semanas para el conjunto de prueba final de CADA cluster
META_MODEL_TYPE = 'ridge'  # Opciones: 'ridge', 'lightgbm', 'rf' (Random Forest)

# Parámetros para control de memoria y procesamiento por lotes
MAX_ROWS_PER_CLUSTER = 1000000  # Límite de filas por clúster (None para sin límite)
CHUNK_SIZE = 5000  # Tamaño de lote para predicciones si los datasets son grandes

# Plantillas para nombres de archivos de modelos base preentrenados
# !! AJUSTA LA EXTENSIÓN (.joblib o .pkl) SEGÚN TUS ARCHIVOS !!
BASE_LGBM_MODEL_NAME_TEMPLATE = 'lgbm_base_cluster_{cluster_id}.joblib'
BASE_RF_MODEL_NAME_TEMPLATE = 'rf_base_cluster_{cluster_id}.joblib'

# --- Creación de Directorios de Salida ---
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(PREDICTIONS_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)

log_progress_initial(f"Directorio de salida principal: {OUTPUT_DIR}")
log_progress_initial(f"Modelos base preentrenados se esperan en: {PRETRAINED_MODELS_DIR}")
log_progress_initial(f"Meta-modelos se guardarán en: {MODELS_DIR}")
log_progress_initial(f"Las predicciones se guardarán en: {PREDICTIONS_DIR}")
log_progress_initial(f"Los gráficos se guardarán en: {PLOTS_DIR}")
log_progress_initial(f"El archivo de métricas será: {METRICS_FILE}")
log_progress_initial(f"El archivo de checkpoint será: {CHECKPOINT_FILE}")
log_progress_initial(f"Meta-modelo a utilizar: {META_MODEL_TYPE.upper()}")


2025-05-15 08:54:20,695 - INFO - [2025-05-15 08:54:20] Iniciando script de modelos de stacking (con modelos base preentrenados) por clusters - v3
2025-05-15 08:54:20,697 - INFO - [2025-05-15 08:54:20] Directorio de salida principal: ../models/stacking_preloaded_v3
2025-05-15 08:54:20,697 - INFO - [2025-05-15 08:54:20] Modelos base preentrenados se esperan en: ../models/stacking/trained_models
2025-05-15 08:54:20,698 - INFO - [2025-05-15 08:54:20] Meta-modelos se guardarán en: ../models/stacking_preloaded_v3/trained_meta_models
2025-05-15 08:54:20,698 - INFO - [2025-05-15 08:54:20] Las predicciones se guardarán en: ../models/stacking_preloaded_v3/predictions
2025-05-15 08:54:20,699 - INFO - [2025-05-15 08:54:20] Los gráficos se guardarán en: ../models/stacking_preloaded_v3/prediction_plots
2025-05-15 08:54:20,699 - INFO - [2025-05-15 08:54:20] El archivo de métricas será: ../models/stacking_preloaded_v3/all_clusters_metrics.csv
2025-05-15 08:54:20,700 - INFO - [2025-05-15 08:54:20] El a

In [8]:
# --- Definiciones de Funciones ---

def log_memory_usage():
    """Registra el uso actual de memoria del proceso."""
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    mem_gb = mem_info.rss / (1024 ** 3)  # Convertir bytes a GB
    logger.info(f"Uso de memoria: {mem_gb:.2f} GB")

def log_progress(message):
    """Registra un mensaje de progreso con timestamp y fuerza la escritura inmediata."""
    timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    logger.info(f"[{timestamp}] {message}")
    sys.stdout.flush()

def load_and_prepare_cluster_data(parquet_path, cluster_id, target_col, date_col, cluster_col_name, series_id_cols, n_test_weeks, max_rows=None):
    """Carga y prepara los datos para un clúster específico."""
    log_progress(f"\n--- Cargando y Preparando Datos para Cluster {cluster_id} ---")
    data_tuple = (None,) * 7 # X_train, y_train, X_test, y_test, cat_features, dates_test, ids_test

    try:
        # Leer solo las columnas necesarias inicialmente para determinar el rango de fechas si max_rows está activo
        initial_cols_to_read = [date_col, cluster_col_name] + series_id_cols + [target_col]
        # Asegurarse que no haya duplicados en initial_cols_to_read
        initial_cols_to_read = sorted(list(set(initial_cols_to_read)))

        filters = [(cluster_col_name, '=', cluster_id)]
        df_initial_check = pd.read_parquet(parquet_path, filters=filters, columns=initial_cols_to_read)
        
        if df_initial_check.empty:
            logger.warning(f"No hay datos para cluster {cluster_id} tras el filtro inicial.")
            return data_tuple
        
        total_rows = len(df_initial_check)
        log_progress(f"Cluster {cluster_id}: Total de {total_rows} filas encontradas inicialmente.")

        cutoff_date_limit = None
        if max_rows is not None and total_rows > max_rows:
            log_progress(f"Limitando a ~{max_rows} filas (de {total_rows}) para control de memoria, seleccionando las más recientes.")
            df_initial_check[date_col] = pd.to_datetime(df_initial_check[date_col])
            df_initial_check = df_initial_check.sort_values(by=date_col, ascending=False)
            cutoff_date_limit = df_initial_check[date_col].iloc[max_rows -1] # -1 porque es 0-indexed
            log_progress(f"Se conservarán datos desde la fecha: {cutoff_date_limit} para el cluster {cluster_id}")
        
        del df_initial_check
        gc.collect()

        # Cargar todas las columnas necesarias para el clúster, aplicando el filtro de fecha si es necesario
        if cutoff_date_limit:
            # Este filtro puede ser lento si se aplica después de leer todo el Parquet.
            # Idealmente, Parquet apoya predicados, pero pd.read_parquet no siempre los pasa bien para > en fechas.
            # Una estrategia es leer por chunks o usar Dask/PyArrow directo si esto es un cuello de botella.
            # Por ahora, se lee todo el cluster y luego se filtra en Pandas.
            df_cluster = pd.read_parquet(parquet_path, filters=filters)
            df_cluster[date_col] = pd.to_datetime(df_cluster[date_col])
            df_cluster = df_cluster[df_cluster[date_col] >= cutoff_date_limit]
            log_progress(f"Filtrado por fecha en pandas: {len(df_cluster)} filas mantenidas desde {cutoff_date_limit}")
        else:
            df_cluster = pd.read_parquet(parquet_path, filters=filters)
        
        log_progress(f"Cluster {cluster_id}: {len(df_cluster)} filas cargadas efectivamente para procesamiento.")
        if df_cluster.empty:
            logger.warning(f"No hay datos para cluster {cluster_id} después de la carga y filtrado de fecha.")
            return data_tuple
        log_memory_usage()

    except Exception as e:
        logger.error(f"Error severo cargando datos para cluster {cluster_id}: {e}", exc_info=True)
        return data_tuple

    if not pd.api.types.is_datetime64_any_dtype(df_cluster[date_col]):
        try:
            df_cluster[date_col] = pd.to_datetime(df_cluster[date_col])
        except Exception as parse_error:
            logger.error(f"Error convirtiendo columna '{date_col}' a datetime para cluster {cluster_id}: {parse_error}.")
            return data_tuple

    df_cluster = df_cluster.sort_values(by=series_id_cols + [date_col]).reset_index(drop=True)

    # Manejo de NaNs en features (asumiendo que las features ya existen en el Parquet)
    initial_rows = len(df_cluster)
    cols_with_potential_nans = [col for col in df_cluster.columns if
                                'lag_' in col or 'roll_' in col or 'ewm' in col or
                                'diff' in col or 'days_since_last_sale' in col or
                                'expanding_' in col or 'shift_' in col]
    cols_to_drop_nans_present = [col for col in cols_with_potential_nans if col in df_cluster.columns]
    
    if cols_to_drop_nans_present:
        df_cluster.dropna(subset=cols_to_drop_nans_present, inplace=True)
    
    final_rows = len(df_cluster)
    if initial_rows > final_rows:
        log_progress(f"Manejo de NaNs en features derivadas: Se eliminaron {initial_rows - final_rows} filas.")
    
    if df_cluster.empty:
        logger.error(f"DataFrame del cluster {cluster_id} vacío después de eliminar NaNs en features.")
        return data_tuple

    # Identificación de features categóricas
    # Esta lista debe ser lo más completa posible, o inferirse de dtypes si es más robusto
    categorical_features_list = [
        'establecimiento', 'material', # Suelen ser categóricas
        'year', 'month', 'week_of_year', 'day_of_week', 'day_of_month', 'day_of_year', 'quarter', # Derivadas de fecha
        'has_promo', 'is_covid_period', 'is_holiday_exact_date', 'is_holiday_in_week' # Indicadores booleanos/categóricos
    ]
    # Mantener solo las que existen en el DataFrame
    categorical_features_present = [col for col in categorical_features_list if col in df_cluster.columns]
    
    for col in categorical_features_present:
        # Convertir a 'category' dtype si no lo son ya. LightGBM lo maneja bien.
        # Para RF, esto significa que usará los códigos de categoría subyacentes si no se hace OHE.
        if df_cluster[col].isnull().any():
             df_cluster[col] = df_cluster[col].fillna('Missing').astype('category') # Rellenar y convertir
        elif not isinstance(df_cluster[col].dtype, pd.CategoricalDtype):
            try:
                df_cluster[col] = df_cluster[col].astype('category')
            except TypeError as e:
                 logger.warning(f"No se pudo convertir '{col}' a category en cluster {cluster_id}: {e}. Se usará como está.")

    # Definición de features (X) y target (y)
    columns_to_exclude = [target_col, date_col, cluster_col_name, 'last_sale_week', 'unique_id'] # unique_id si existe
    features = [col for col in df_cluster.columns if col not in columns_to_exclude]
    X = df_cluster[features]
    y = df_cluster[target_col]

    # Split Train/Test basado en tiempo
    if len(df_cluster) < (n_test_weeks + 1): # Necesitas al menos 1 para entrenar y n_test_weeks para testear
         logger.warning(f"Datos insuficientes en cluster {cluster_id} ({len(df_cluster)} filas) para split con {n_test_weeks} semanas de test.")
         return data_tuple

    last_date_in_cluster = df_cluster[date_col].max()
    # El cutoff es la primera fecha del conjunto de test
    cutoff_date_for_test_start = last_date_in_cluster - pd.Timedelta(weeks=n_test_weeks - 1) 
    
    train_mask = (df_cluster[date_col] < cutoff_date_for_test_start)
    test_mask = (df_cluster[date_col] >= cutoff_date_for_test_start)

    if train_mask.sum() == 0:
        logger.warning(f"Conjunto de entrenamiento vacío después del split en cluster {cluster_id} (cutoff: {cutoff_date_for_test_start}).")
        return data_tuple
    if test_mask.sum() == 0:
        logger.warning(f"Conjunto de prueba vacío después del split en cluster {cluster_id} (cutoff: {cutoff_date_for_test_start}).")
        return data_tuple

    X_train, X_test = X[train_mask].copy(), X[test_mask].copy()
    y_train, y_test = y[train_mask].copy(), y[test_mask].copy()
    dates_test = df_cluster.loc[test_mask, date_col].copy()
    ids_test = df_cluster.loc[test_mask, series_id_cols].copy()

    del df_cluster # Liberar memoria
    gc.collect()
    log_memory_usage()

    log_progress(f"Cluster {cluster_id} Split: Train={X_train.shape[0]}, Test={X_test.shape[0]}. Features categóricas identificadas: {categorical_features_present}")
    return X_train, y_train, X_test, y_test, categorical_features_present, dates_test, ids_test


def generate_base_model_predictions_cluster(X_train, X_test, cluster_id, pretrained_models_path, lgbm_template, rf_template, cat_features_for_lgbm):
    """Carga modelos LGBM y RF preentrenados y genera predicciones."""
    log_progress(f"--- Cargando Modelos Base y Generando Predicciones para Cluster {cluster_id} ---")
    predictions_tuple = (None, None, None, None) # oof_lgbm, oof_rf, test_lgbm, test_rf

    lgbm_model_file = os.path.join(pretrained_models_path, lgbm_template.format(cluster_id=cluster_id))
    rf_model_file = os.path.join(pretrained_models_path, rf_template.format(cluster_id=cluster_id))

    base_model_lgbm, base_model_rf = None, None
    try:
        log_progress(f"Cargando modelo LightGBM desde: {lgbm_model_file}")
        base_model_lgbm = joblib.load(lgbm_model_file)
    except FileNotFoundError:
        logger.error(f"Modelo LightGBM NO ENCONTRADO para cluster {cluster_id} en {lgbm_model_file}")
        return predictions_tuple
    except Exception as e:
        logger.error(f"Error cargando modelo LightGBM para cluster {cluster_id}: {e}", exc_info=True)
        return predictions_tuple

    try:
        log_progress(f"Cargando modelo Random Forest desde: {rf_model_file}")
        base_model_rf = joblib.load(rf_model_file)
    except FileNotFoundError:
        logger.error(f"Modelo Random Forest NO ENCONTRADO para cluster {cluster_id} en {rf_model_file}")
        return predictions_tuple
    except Exception as e:
        logger.error(f"Error cargando modelo Random Forest para cluster {cluster_id}: {e}", exc_info=True)
        return predictions_tuple

    # Asegurar que X_train y X_test tengan las features categóricas como 'category' para LGBM si es necesario
    # Esta conversión ya se hace en load_and_prepare_cluster_data, pero un check doble no hace daño.
    # X_train_lgbm_ready = X_train.copy()
    # X_test_lgbm_ready = X_test.copy()
    # for col in cat_features_for_lgbm:
    #     if col in X_train_lgbm_ready.columns and not isinstance(X_train_lgbm_ready[col].dtype, pd.CategoricalDtype):
    #         X_train_lgbm_ready[col] = X_train_lgbm_ready[col].astype('category')
    #     if col in X_test_lgbm_ready.columns and not isinstance(X_test_lgbm_ready[col].dtype, pd.CategoricalDtype):
    #         X_test_lgbm_ready[col] = X_test_lgbm_ready[col].astype('category')
    
    # Predicciones "OOF" (directas sobre el conjunto de entrenamiento)
    log_progress(f"Generando predicciones (tipo OOF) en datos de entrenamiento para cluster {cluster_id}")
    # Para RF, se asume que las features categóricas ya están codificadas como esperaba el modelo (ej. label encoded por .cat.codes)
    # o que el modelo RF puede manejarlas si son 'category'. Sklearn RF no las usa directamente.
    # Si RF fue entrenado con OHE, X_train/X_test necesitarían esa OHE aquí.
    try:
        oof_preds_lgbm_vals = base_model_lgbm.predict(X_train) # Usar X_train directamente
        oof_preds_rf_vals = base_model_rf.predict(X_train)
    except Exception as e:
        logger.error(f"Error generando predicciones OOF para cluster {cluster_id}: {e}", exc_info=True)
        logger.error(f"Columnas X_train: {X_train.columns.tolist()}")
        logger.error(f"Tipos X_train: {X_train.dtypes}")
        return predictions_tuple


    oof_series_lgbm = pd.Series(oof_preds_lgbm_vals, index=X_train.index, name='oof_lgbm_pred')
    oof_series_rf = pd.Series(oof_preds_rf_vals, index=X_train.index, name='oof_rf_pred')

    # Predicciones en el conjunto de prueba
    log_progress(f"Generando predicciones en datos de prueba para cluster {cluster_id}")
    try:
        if len(X_test) > CHUNK_SIZE:
            log_progress(f"Prediciendo en lotes (chunk_size={CHUNK_SIZE}) para Test (Cluster {cluster_id})")
            test_preds_lgbm = np.concatenate([
                base_model_lgbm.predict(X_test.iloc[i:i + CHUNK_SIZE]) 
                for i in range(0, len(X_test), CHUNK_SIZE)
            ])
            test_preds_rf = np.concatenate([
                base_model_rf.predict(X_test.iloc[i:i + CHUNK_SIZE])
                for i in range(0, len(X_test), CHUNK_SIZE)
            ])
        else:
            test_preds_lgbm = base_model_lgbm.predict(X_test) # Usar X_test directamente
            test_preds_rf = base_model_rf.predict(X_test)
    except Exception as e:
        logger.error(f"Error generando predicciones en Test para cluster {cluster_id}: {e}", exc_info=True)
        logger.error(f"Columnas X_test: {X_test.columns.tolist()}")
        logger.error(f"Tipos X_test: {X_test.dtypes}")

        return predictions_tuple
        
    del base_model_lgbm, base_model_rf
    gc.collect()
    log_memory_usage()
    return oof_series_lgbm, oof_series_rf, test_preds_lgbm, test_preds_rf

def train_meta_model_cluster(X_train_original_features, oof_preds_lgbm, oof_preds_rf, y_train, cluster_id, meta_model_type, models_save_dir):
    """Entrena el meta-modelo para un clúster."""
    log_progress(f"--- Entrenando Meta-Modelo ({meta_model_type.upper()}) para Cluster {cluster_id} ---")
    
    # Construir el DataFrame para entrenar el meta-modelo
    # Opción 1: Usar solo las predicciones OOF como features
    # X_meta_train = pd.DataFrame({
    #     'oof_lgbm': oof_preds_lgbm.values,
    #     'oof_rf': oof_preds_rf.values
    # }, index=oof_preds_lgbm.index)

    # Opción 2: Usar las features originales + las predicciones OOF
    X_meta_train = pd.DataFrame({
        'oof_lgbm_pred_meta': oof_preds_lgbm.values,
        'oof_rf_pred_meta': oof_preds_rf.values
    }, index=X_train_original_features.index) # Usar el índice original para consistencia con y_train

    meta_model = None
    try:
        if meta_model_type == 'ridge':
            categorical_cols_in_meta = X_meta_train.select_dtypes(include='category').columns
            if not categorical_cols_in_meta.empty:
                log_progress(f"Aplicando get_dummies a X_meta_train para Ridge: {categorical_cols_in_meta.tolist()}")
                X_meta_train = pd.get_dummies(X_meta_train, columns=categorical_cols_in_meta, dummy_na=False, sparse=True)
            meta_model = Ridge(alpha=1.0, random_state=42)

        elif meta_model_type == 'lightgbm':
            meta_params_lgbm = {
                'objective': 'regression_l1', 'metric': 'mae', 'verbosity': -1, 'boosting_type': 'gbdt',
                'seed': 42, 'n_jobs': -1, 'learning_rate': 0.05, 'num_leaves': 25, 
                'max_depth': 5, 'n_estimators': 200, 'feature_fraction': 0.8, 'bagging_fraction': 0.8,
                'bagging_freq': 1, 'min_child_samples': 30
            }
            meta_model = lgb.LGBMRegressor(**meta_params_lgbm)
            # Identificar features categóricas para LGBM meta (excluyendo las OOF que son numéricas)
            # Las features originales que eran categóricas deben pasarse a LGBM
            cat_features_for_meta_lgbm = [
                col for col in X_meta_train.select_dtypes(include='category').columns
                if col not in ['oof_lgbm_pred_meta', 'oof_rf_pred_meta']
            ]
            if not cat_features_for_meta_lgbm: cat_features_for_meta_lgbm = 'auto'
            
            log_progress(f"Entrenando meta-modelo LightGBM con features categóricas: {cat_features_for_meta_lgbm}")
            meta_model.fit(X_meta_train, y_train, categorical_feature=cat_features_for_meta_lgbm)
            # Early stopping podría añadirse aquí si se define un eval_set para el meta-modelo

        elif meta_model_type == 'rf':
            categorical_cols_in_meta_rf = X_meta_train.select_dtypes(include='category').columns
            if not categorical_cols_in_meta_rf.empty:
                log_progress(f"Aplicando get_dummies a X_meta_train para RF: {categorical_cols_in_meta_rf.tolist()}")
                X_meta_train = pd.get_dummies(X_meta_train, columns=categorical_cols_in_meta_rf, dummy_na=False)
            
            meta_params_rf = {
                'n_estimators': 150, 'random_state': 42, 'n_jobs': -1, 'max_depth': 10,
                'min_samples_split': 40, 'min_samples_leaf': 25, 'max_features': 0.7
            }
            meta_model = RandomForestRegressor(**meta_params_rf)
        else:
            logger.error(f"Tipo de meta-modelo '{meta_model_type}' no soportado.")
            return None

        # Entrenar (si no es LGBM que ya se entrenó con sus particularidades)
        if meta_model_type != 'lightgbm': # LGBM ya se entrenó
             # Si X_meta_train es sparse (por Ridge) y el modelo no lo maneja, convertir a denso
             # if hasattr(X_meta_train, "toarray") and meta_model_type not in ['lightgbm']: # lightgbm maneja sparse
             #     X_meta_train = X_meta_train.toarray()
            meta_model.fit(X_meta_train, y_train)

        log_progress(f"Meta-modelo ({meta_model_type.upper()}) para cluster {cluster_id} entrenado.")
        
        model_meta_path = os.path.join(models_save_dir, f'meta_model_{meta_model_type}_cluster_{cluster_id}.joblib')
        joblib.dump(meta_model, model_meta_path)
        log_progress(f"Meta-modelo guardado en: {model_meta_path}")
        
        return meta_model

    except Exception as e:
        logger.error(f"¡ERROR entrenando el meta-modelo ({meta_model_type.upper()}) para Cluster {cluster_id}!: {e}", exc_info=True)
        if 'X_meta_train' in locals():
            logger.error(f"X_meta_train dtypes: \n{X_meta_train.dtypes.value_counts() if isinstance(X_meta_train, pd.DataFrame) else type(X_meta_train)}")
        return None
    finally:
        if 'X_meta_train' in locals(): del X_meta_train # Asegurar liberación
        gc.collect()
        log_memory_usage()

def evaluate_model_cluster(y_true, y_pred, cluster_id, model_label="Stacking"):
    """Evalúa el modelo y devuelve un diccionario de métricas."""
    log_progress(f"--- Evaluando Modelo {model_label} para Cluster {cluster_id} ---")
    
    y_true_safe = np.asarray(y_true)
    y_pred_safe = np.asarray(y_pred)
    
    # Reemplazar NaNs/Infs en predicciones si los hubiera
    if np.isnan(y_pred_safe).any() or np.isinf(y_pred_safe).any():
        logger.warning(f"Cluster {cluster_id}, Modelo {model_label}: Predicciones contienen NaNs/Infs. Se reemplazarán con 0 para evaluación.")
        y_pred_safe = np.nan_to_num(y_pred_safe, nan=0.0, posinf=np.nanmax(y_true_safe) if y_true_safe.size > 0 else 0.0 , neginf=0.0)
    
    rmse = np.sqrt(mean_squared_error(y_true_safe, y_pred_safe))
    mae = mean_absolute_error(y_true_safe, y_pred_safe)
    r2 = r2_score(y_true_safe, y_pred_safe)  # <--- CALCULAR R-CUADRADO

    log_progress(f"  {model_label} - MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}") 
    return {'cluster_id': cluster_id, f'{model_label}_mae': mae, f'{model_label}_rmse': rmse, f'{model_label}_r2': r2}

def plot_predictions_series(dates, y_true, y_pred, estab, material, cluster_id, save_path, model_label="Stacking"):
    """Genera y guarda un gráfico de predicciones vs reales para una serie."""
    plt.figure(figsize=(15, 6))
    plt.plot(dates, y_true, label='Real', marker='.', linestyle='-', color='dodgerblue', alpha=0.8)
    plt.plot(dates, y_pred, label=f'Predicción {model_label}', marker='x', linestyle='--', color='orangered', alpha=0.8)
    plt.title(f'Predicción vs Real - Est: {estab}, Mat: {material} (Clúster {cluster_id})', fontsize=14)
    plt.xlabel('Semana', fontsize=12)
    plt.ylabel('Volumen Semanal', fontsize=12)
    plt.legend(fontsize=10)
    plt.grid(True, linestyle=':', alpha=0.6)
    plt.xticks(rotation=45)
    plt.tight_layout()
    try:
        plt.savefig(save_path, dpi=100)
    except Exception as e:
        logger.error(f"Error guardando gráfico {save_path} para cluster {cluster_id}: {e}")
    plt.close() # Cerrar la figura para liberar memoria
    gc.collect()

In [10]:

master_start_time = time.time()
log_progress("================ INICIO DEL PROCESAMIENTO DE STACKING ================")

try:
    log_progress(f"Leyendo columna de clusters '{CLUSTER_COL}' del archivo Parquet: {PARQUET_FILE_PATH}")
    # Leer solo la columna de clusters para eficiencia
    cluster_labels_df = pd.read_parquet(PARQUET_FILE_PATH, columns=[CLUSTER_COL])
    unique_clusters = sorted([c for c in cluster_labels_df[CLUSTER_COL].unique() if pd.notna(c)])
    del cluster_labels_df
    gc.collect()
    if not unique_clusters:
        log_progress("No se encontraron clusters únicos en el archivo. Terminando script.")
        exit()
    log_progress(f"Encontrados {len(unique_clusters)} clusters únicos para procesar: {unique_clusters}")
except Exception as e:
    logger.error(f"Error crítico al leer la columna de clusters del Parquet: {e}", exc_info=True)
    exit()

processed_clusters_metrics = []
clusters_successfully_processed_count = 0
clusters_skipped_or_failed_count = 0

# Crear archivo de métricas con encabezado si no existe o está vacío
if not os.path.exists(METRICS_FILE) or os.path.getsize(METRICS_FILE) == 0:
    pd.DataFrame(columns=['cluster_id', 'Stacking_mae', 'Stacking_rmse', 'Stacking_r2']).to_csv(METRICS_FILE, index=False)
    log_progress(f"Archivo de métricas {METRICS_FILE} creado con encabezado.")

# Lógica de Checkpoint para reanudar
last_successfully_processed_cluster_idx = -1
if os.path.exists(CHECKPOINT_FILE):
    try:
        with open(CHECKPOINT_FILE, 'r') as f_check:
            checkpoint_content = f_check.read().strip()
            if checkpoint_content:
                last_successfully_processed_cluster_idx = int(checkpoint_content)
                log_progress(f"Reanudando desde checkpoint: último índice de clúster procesado con éxito = {last_successfully_processed_cluster_idx}")
    except Exception as e_check:
        logger.error(f"Error leyendo archivo de checkpoint ({CHECKPOINT_FILE}): {e_check}. Se procesarán todos los clusters.", exc_info=True)
        last_successfully_processed_cluster_idx = -1 # Resetear si hay error

# --- Bucle Principal por Cluster ---
for current_cluster_idx, cluster_id_val in enumerate(unique_clusters):
    if current_cluster_idx <= last_successfully_processed_cluster_idx:
        log_progress(f"Saltando Clúster {cluster_id_val} (índice {current_cluster_idx}), ya procesado según checkpoint.")
        continue

    cluster_loop_start_time = time.time()
    log_progress(f"\n========== Procesando Cluster {current_cluster_idx + 1}/{len(unique_clusters)}: ID = {cluster_id_val} ==========")
    log_memory_usage()
    
    cluster_processed_flag = False
    try:
        # 1. Cargar y preparar datos para el cluster
        X_train_c, y_train_c, X_test_c, y_test_c, cat_features_c, dates_test_c, ids_test_c = \
            load_and_prepare_cluster_data(
                PARQUET_FILE_PATH, cluster_id_val, TARGET_COL, DATE_COL, CLUSTER_COL,
                SERIES_ID_COLS, N_TEST_WEEKS, max_rows=MAX_ROWS_PER_CLUSTER
            )

        if X_train_c is None or X_test_c is None or y_train_c is None or y_test_c is None:
            log_progress(f"Datos insuficientes o error en carga para cluster {cluster_id_val}. Saltando.")
            clusters_skipped_or_failed_count += 1
            # Guardar checkpoint para marcar este cluster como "intentado" y no reintentar si falla consistentemente
            with open(CHECKPOINT_FILE, 'w') as f_checkpoint_skip:
                f_checkpoint_skip.write(str(current_cluster_idx))
            continue

        # 2. Generar predicciones de modelos base preentrenados
        oof_lgbm, oof_rf, test_lgbm, test_rf = generate_base_model_predictions_cluster(
            X_train_c, X_test_c, cluster_id_val, PRETRAINED_MODELS_DIR,
            BASE_LGBM_MODEL_NAME_TEMPLATE, BASE_RF_MODEL_NAME_TEMPLATE, cat_features_c
        )

        if oof_lgbm is None or oof_rf is None or test_lgbm is None or test_rf is None:
            log_progress(f"Fallo en generación de predicciones base para cluster {cluster_id_val}. Saltando.")
            clusters_skipped_or_failed_count += 1
            with open(CHECKPOINT_FILE, 'w') as f_checkpoint_skip:
                f_checkpoint_skip.write(str(current_cluster_idx))
            continue

        # 3. Entrenar meta-modelo
        meta_model_trained = train_meta_model_cluster(
            X_train_c, oof_lgbm, oof_rf, y_train_c, cluster_id_val, META_MODEL_TYPE, MODELS_DIR
        )

        if meta_model_trained is None:
            log_progress(f"Fallo en entrenamiento del meta-modelo para cluster {cluster_id_val}. Saltando.")
            clusters_skipped_or_failed_count += 1
            with open(CHECKPOINT_FILE, 'w') as f_checkpoint_skip:
                f_checkpoint_skip.write(str(current_cluster_idx))
            continue

        # 4. Preparar datos de Test para el Meta-Modelo y Predecir
        log_progress("Construyendo X_meta_test_final usando SOLO las predicciones de prueba de los modelos base.")
        X_meta_test_final = pd.DataFrame({
            'oof_lgbm_pred_meta': test_lgbm, # Usar el mismo nombre que en entrenamiento del meta
            'oof_rf_pred_meta': test_rf    # Usar el mismo nombre que en entrenamiento del meta
        }, index=X_test_c.index) # Mantener el índice de X_test_c para posible alineación futura si es necesario
        # Aplicar transformaciones a X_meta_test_final si es necesario (ej. get_dummies, reindexar)
        # Esto debe reflejar exactamente lo que se hizo en train_meta_model_cluster
        if META_MODEL_TYPE == 'ridge' or META_MODEL_TYPE == 'rf':
            cat_cols_meta_test = X_meta_test_final.select_dtypes(include='category').columns
            if not cat_cols_meta_test.empty:
                X_meta_test_final = pd.get_dummies(X_meta_test_final, columns=cat_cols_meta_test, dummy_na=False, 
                                                    sparse=(META_MODEL_TYPE == 'ridge')) # sparse solo para ridge si se usó
            
            # Reindexar para asegurar consistencia de columnas con el entrenamiento
            if hasattr(meta_model_trained, 'feature_names_in_'):
                train_cols = meta_model_trained.feature_names_in_
                X_meta_test_final = X_meta_test_final.reindex(columns=train_cols, fill_value=0)
            else:
                logger.warning(f"Meta-modelo ({META_MODEL_TYPE}) no tiene 'feature_names_in_'. No se puede reindexar X_meta_test_final para cluster {cluster_id_val}.")
        
        # Para LGBM meta-modelo, las categóricas ya deberían estar como 'category'
        # No se necesita get_dummies, pero asegurar que los tipos son correctos.
        
        # Imputar NaNs que pudieron surgir de reindexación o por otras razones
        if isinstance(X_meta_test_final, pd.DataFrame):
            X_meta_test_final.fillna(0, inplace=True) # Imputación simple
        elif hasattr(X_meta_test_final, 'fillna'): # Pandas SparseDataFrame
                X_meta_test_final = X_meta_test_final.fillna(0)


        log_progress(f"Generando predicciones finales Stacking para cluster {cluster_id_val}")
        stacking_final_predictions = meta_model_trained.predict(X_meta_test_final)
        stacking_final_predictions = np.maximum(0, stacking_final_predictions) # Asegurar no-negatividad

        # 5. Evaluar y Guardar Resultados
        cluster_metrics = evaluate_model_cluster(y_test_c.values, stacking_final_predictions, cluster_id_val)
        processed_clusters_metrics.append(cluster_metrics)
        pd.DataFrame([cluster_metrics]).to_csv(METRICS_FILE, mode='a', header=False, index=False)

        # Guardar predicciones detalladas
        preds_df = pd.DataFrame({
            'establecimiento': ids_test_c['establecimiento'], 'material': ids_test_c['material'],
            'week': dates_test_c, 'actual_volume': y_test_c.values,
            'stacking_predicted_volume': stacking_final_predictions, 'cluster_label': cluster_id_val
        }).sort_values(by=SERIES_ID_COLS + ['week']).reset_index(drop=True)
        
        pred_file_path = os.path.join(PREDICTIONS_DIR, f'stacking_preds_cluster_{cluster_id_val}.csv')
        preds_df.to_csv(pred_file_path, index=False)
        log_progress(f"Predicciones para cluster {cluster_id_val} guardadas en: {pred_file_path}")

        # 6. Generar Gráficos (para una muestra de series)
        log_progress(f"Generando gráficos para series del cluster {cluster_id_val}")
        unique_series_to_plot = ids_test_c.drop_duplicates().head(min(5, len(ids_test_c))).values.tolist() # Max 5 plots
        for series_components in unique_series_to_plot:
            estab_val, mat_val = series_components[0], series_components[1]
            series_mask = (preds_df['establecimiento'] == estab_val) & (preds_df['material'] == mat_val)
            series_data_for_plot = preds_df[series_mask]
            if not series_data_for_plot.empty:
                plot_file = os.path.join(PLOTS_DIR, f'plot_est{estab_val}_mat{mat_val}_clust{cluster_id_val}.png')
                plot_predictions_series(
                    series_data_for_plot['week'], series_data_for_plot['actual_volume'],
                    series_data_for_plot['stacking_predicted_volume'],
                    estab_val, mat_val, cluster_id_val, plot_file
                )
        cluster_processed_flag = True # Marcador de éxito para este clúster

    except Exception as e_cluster_loop:
        logger.error(f"ERROR INESPERADO procesando cluster {cluster_id_val} (índice {current_cluster_idx}): {e_cluster_loop}", exc_info=True)
        clusters_skipped_or_failed_count += 1
    finally:
        # Actualizar checkpoint
        if cluster_processed_flag:
            clusters_successfully_processed_count += 1
            with open(CHECKPOINT_FILE, 'w') as f_checkpoint:
                f_checkpoint.write(str(current_cluster_idx))
            log_progress(f"Checkpoint actualizado a índice {current_cluster_idx} (Cluster {cluster_id_val} procesado con éxito).")
        # else: # Si falló, el checkpoint ya se guardó dentro del try o no se guarda para reintentar la próxima vez
        #    log_progress(f"Cluster {cluster_id_val} (índice {current_cluster_idx}) no se procesó con éxito. Checkpoint no avanzado por esta razón.")


        # Limpieza de memoria al final de cada iteración del bucle del clúster
        vars_to_del_loop = ['X_train_c', 'y_train_c', 'X_test_c', 'y_test_c', 'cat_features_c', 
                            'dates_test_c', 'ids_test_c', 'oof_lgbm', 'oof_rf', 'test_lgbm', 'test_rf',
                            'meta_model_trained', 'X_meta_test_final', 'stacking_final_predictions', 'preds_df']
        for var_name_del in vars_to_del_loop:
            if var_name_del in locals():
                del locals()[var_name_del]
        gc.collect()
        
        cluster_loop_time = time.time() - cluster_loop_start_time
        log_progress(f"--- Cluster {cluster_id_val} (índice {current_cluster_idx}) finalizado. Tiempo: {cluster_loop_time:.2f} seg. ---")
        log_memory_usage()

        # Estimación de tiempo restante
        # current_processed_in_this_run = (current_cluster_idx - last_successfully_processed_cluster_idx)
        # if current_processed_in_this_run > 0:
        #    time_elapsed_this_run = time.time() - master_start_time # O un timer específico para esta corrida
        #    avg_time_per_cluster_this_run = time_elapsed_this_run / current_processed_in_this_run
        #    remaining_clusters = len(unique_clusters) - (current_cluster_idx + 1)
        #    est_time_left = avg_time_per_cluster_this_run * remaining_clusters
        #    log_progress(f"Tiempo promedio por cluster (esta ejecución): {avg_time_per_cluster_this_run:.2f} seg.")
        #    log_progress(f"Tiempo restante estimado: {est_time_left / 60:.2f} min / {est_time_left / 3600:.2f} hrs.")


# --- Finalización del Script ---
master_total_time = time.time() - master_start_time
log_progress("\n====================== PROCESAMIENTO DE TODOS LOS CLUSTERS COMPLETADO ======================")
log_progress(f"Tiempo total de ejecución del script: {master_total_time / 60:.2f} minutos ({master_total_time / 3600:.2f} horas).")
log_progress(f"Total de clusters configurados: {len(unique_clusters)}")
log_progress(f"Clusters procesados con éxito: {clusters_successfully_processed_count}")
log_progress(f"Clusters saltados o fallidos: {clusters_skipped_or_failed_count}")

log_progress(f"Resultados guardados en el directorio base: {OUTPUT_DIR}")
log_progress(f"Métricas consolidadas por cluster en: {METRICS_FILE}")

try:
    if os.path.exists(METRICS_FILE) and os.path.getsize(METRICS_FILE) > 0:
        final_metrics_df = pd.read_csv(METRICS_FILE)
        if not final_metrics_df.empty and 'Stacking_mae' in final_metrics_df.columns and 'Stacking_rmse' in final_metrics_df.columns and 'Stacking_r2' in final_metrics_df.columns:
            log_progress("\n--- Resumen Métricas Finales Agregadas (sobre clusters procesados exitosamente) ---")
            log_progress(f"MAE Promedio Stacking: {final_metrics_df['Stacking_mae'].mean():.4f}")
            log_progress(f"RMSE Promedio Stacking: {final_metrics_df['Stacking_rmse'].mean():.4f}")
            log_progress(f"R2 Promedio Stacking: {final_metrics_df['Stacking_r2'].mean():.4f}")
        else:
            log_progress("No se pudieron calcular métricas promedio (archivo vacío o columnas faltantes).")
    else:
        log_progress("Archivo de métricas no encontrado o vacío. No hay resumen de promedio.")
except Exception as e_summary:
    logger.error(f"Error al generar el resumen de métricas: {e_summary}", exc_info=True)

# Eliminar archivo checkpoint si todos los clusters fueron procesados (o intentados)
total_attempted_clusters = last_successfully_processed_cluster_idx + 1 + clusters_successfully_processed_count + clusters_skipped_or_failed_count
# Esta lógica de conteo para eliminación de checkpoint puede ser compleja si se reanuda.
# Simplificación: si el último índice procesado es el último de la lista.
if (last_successfully_processed_cluster_idx + clusters_successfully_processed_count + clusters_skipped_or_failed_count) >= (len(unique_clusters) -1) :
        if os.path.exists(CHECKPOINT_FILE):
            log_progress("Todos los clusters han sido abordados. Eliminando archivo checkpoint.")
        try:
            os.remove(CHECKPOINT_FILE)
        except OSError as e_rm_check:
            logger.error(f"No se pudo eliminar el archivo checkpoint ({CHECKPOINT_FILE}): {e_rm_check}")
else:
    log_progress(f"El archivo checkpoint {CHECKPOINT_FILE} se mantiene, ya que no todos los clusters parecen haber sido procesados en esta o previas ejecuciones.")

log_progress("========================== SCRIPT DE STACKING FINALIZADO ==========================")

2025-05-15 08:57:46,973 - INFO - [2025-05-15 08:57:46] ================ INICIO DEL PROCESAMIENTO DE STACKING ================
2025-05-15 08:57:46,975 - INFO - [2025-05-15 08:57:46] Leyendo columna de clusters 'cluster_label' del archivo Parquet: ../data/gold_ventas_semanales_clustered_lgbm.parquet
2025-05-15 08:57:47,199 - INFO - [2025-05-15 08:57:47] Encontrados 7 clusters únicos para procesar: [np.int32(0), np.int32(1), np.int32(2), np.int32(3), np.int32(4), np.int32(5), np.int32(6)]
2025-05-15 08:57:47,201 - INFO - [2025-05-15 08:57:47] Archivo de métricas ../models/stacking_preloaded_v3/all_clusters_metrics.csv creado con encabezado.
2025-05-15 08:57:47,202 - INFO - [2025-05-15 08:57:47] 
========== Procesando Cluster 1/7: ID = 0 ==========
2025-05-15 08:57:47,203 - INFO - Uso de memoria: 1.51 GB
2025-05-15 08:57:47,203 - INFO - [2025-05-15 08:57:47] 
--- Cargando y Preparando Datos para Cluster 0 ---
2025-05-15 08:57:47,505 - INFO - [2025-05-15 08:57:47] Cluster 0: Total de 770252